In [1]:
import pyexasol
import pandas as pd
import boto3
import time
import io
import numpy as np
from matplotlib import pyplot as plt


dwh_host = '***'
dwh_port = '8563'
dwh_user = '***'
dwh_pass = '***'

#connect to db
dwh_conn = pyexasol.connect(dsn=dwh_host + ':' + dwh_port, user=dwh_user, password=dwh_pass)

## data function to get iterator data

In [5]:
#get iterator month data
def get_feature_types():

    #get iterator datam
    
    v_sql = """
SELECT distinct
        feature_type
from
    betting_mart.ml_poisson_features
where
    full_time_home_goals is not null
    and home_shots_for_ema is not null
    and away_shots_against_ema is not null
    """
    
    return dwh_conn.export_to_pandas(v_sql)


#get iterator month data
def get_iterator_data(feature_type_filter):

    #get iterator datam
    
    v_sql = """SELECT distinct
        to_char(match_date,'yyyymm') iterator
    FROM 
        betting_mart.ml_poisson_features
    WHERE
        --2014_15 season dropped for stable features
        season <> '2014_2015'
        --2015_16 is first training season
        AND season <> '2015_2016' and
        --feature type filter
        feature_type = '""" + str(feature_type_filter) + """' and
        --ohne null values
        home_goals_for_ema is not null and
        home_goals_against_ema is not null and
        away_goals_for_ema is not null and
        away_goals_against_ema is not null and
        home_xg_for_ema is not null and
        home_xg_against_ema is not null and
        away_xg_for_ema is not null and
        away_xg_against_ema is not null and
        home_shots_for_ema is not null and
        home_shots_against_ema is not null and
        away_shots_for_ema is not null and
        away_shots_against_ema is not null and
        home_sot_for_ema is not null and
        home_sot_against_ema is not null and
        away_sot_for_ema is not null and
        away_sot_against_ema is not null and
        home_corners_for_ema is not null and
        home_corners_against_ema is not null and
        away_corners_for_ema is not null and
        away_corners_against_ema is not null and
        home_deep_for_ema is not null and
        home_deep_against_ema is not null and
        away_deep_for_ema is not null and
        away_deep_against_ema is not null and
        home_ppda_for_ema is not null and
        home_ppda_against_ema is not null and
        away_ppda_for_ema is not null and
        away_ppda_against_ema is not null and
        full_time_home_goals is not null and
        full_time_away_goals is not null
    ORDER BY 1"""

    return dwh_conn.export_to_pandas(v_sql)


def get_home_training_data(feature_type_filter, month_filter):

    v_sql = """
select
    round(home_xg_for_ema,1)               home_xg_for_ema,    
    round(away_xg_against_ema,1)           away_xg_against_ema,    
    full_time_home_goals                   full_time_home_goals
from
    betting_mart.ml_poisson_features
where
    --filter for iterator
    to_char(match_date,'yyyymm') < """ + str(month_filter) + """ and
    feature_type = '""" + str(feature_type_filter) + """' and
    home_goals_for_ema is not null and
    home_goals_against_ema is not null and
    away_goals_for_ema is not null and
    away_goals_against_ema is not null and
    home_xg_for_ema is not null and
    home_xg_against_ema is not null and
    away_xg_for_ema is not null and
    away_xg_against_ema is not null and
    home_shots_for_ema is not null and
    home_shots_against_ema is not null and
    away_shots_for_ema is not null and
    away_shots_against_ema is not null and
    home_sot_for_ema is not null and
    home_sot_against_ema is not null and
    away_sot_for_ema is not null and
    away_sot_against_ema is not null and
    home_corners_for_ema is not null and
    home_corners_against_ema is not null and
    away_corners_for_ema is not null and
    away_corners_against_ema is not null and
    home_deep_for_ema is not null and
    home_deep_against_ema is not null and
    away_deep_for_ema is not null and
    away_deep_against_ema is not null and
    home_ppda_for_ema is not null and
    home_ppda_against_ema is not null and
    away_ppda_for_ema is not null and
    away_ppda_against_ema is not null and
    full_time_home_goals is not null and
    full_time_away_goals is not null
    """
    
    
    return dwh_conn.export_to_pandas(v_sql)


def get_away_training_data(feature_type_filter, month_filter):

    v_sql = """
select
    round(home_xg_against_ema,1)           home_xg_against_ema,
    round(away_xg_for_ema,1)               away_xg_for_ema,
    full_time_away_goals              full_time_away_goals
from
    betting_mart.ml_poisson_features
where
    --filter for iterator
    to_char(match_date,'yyyymm') < """ + str(month_filter) + """ and
    feature_type = '""" + str(feature_type_filter) + """' and
    home_goals_for_ema is not null and
    home_goals_against_ema is not null and
    away_goals_for_ema is not null and
    away_goals_against_ema is not null and
    home_xg_for_ema is not null and
    home_xg_against_ema is not null and
    away_xg_for_ema is not null and
    away_xg_against_ema is not null and
    home_shots_for_ema is not null and
    home_shots_against_ema is not null and
    away_shots_for_ema is not null and
    away_shots_against_ema is not null and
    home_sot_for_ema is not null and
    home_sot_against_ema is not null and
    away_sot_for_ema is not null and
    away_sot_against_ema is not null and
    home_corners_for_ema is not null and
    home_corners_against_ema is not null and
    away_corners_for_ema is not null and
    away_corners_against_ema is not null and
    home_deep_for_ema is not null and
    home_deep_against_ema is not null and
    away_deep_for_ema is not null and
    away_deep_against_ema is not null and
    home_ppda_for_ema is not null and
    home_ppda_against_ema is not null and
    away_ppda_for_ema is not null and
    away_ppda_against_ema is not null and
    full_time_home_goals is not null and
    full_time_away_goals is not null
    """
    
    
    return dwh_conn.export_to_pandas(v_sql)



def get_home_prediction_data(feature_type_filter, month_filter):
    
    v_sql = """
select
    football_match_his_lid,
    round(home_xg_for_ema,1)               home_xg_for_ema,    
    round(away_xg_against_ema,1)           away_xg_against_ema
from
    betting_mart.ml_poisson_features
where
    --filter for iterator
    to_char(match_date,'yyyymm') = """ + str(month_filter) + """ and
    feature_type = '""" + str(feature_type_filter) + """' and
    home_goals_for_ema is not null and
    home_goals_against_ema is not null and
    away_goals_for_ema is not null and
    away_goals_against_ema is not null and
    home_xg_for_ema is not null and
    home_xg_against_ema is not null and
    away_xg_for_ema is not null and
    away_xg_against_ema is not null and
    home_shots_for_ema is not null and
    home_shots_against_ema is not null and
    away_shots_for_ema is not null and
    away_shots_against_ema is not null and
    home_sot_for_ema is not null and
    home_sot_against_ema is not null and
    away_sot_for_ema is not null and
    away_sot_against_ema is not null and
    home_corners_for_ema is not null and
    home_corners_against_ema is not null and
    away_corners_for_ema is not null and
    away_corners_against_ema is not null and
    home_deep_for_ema is not null and
    home_deep_against_ema is not null and
    away_deep_for_ema is not null and
    away_deep_against_ema is not null and
    home_ppda_for_ema is not null and
    home_ppda_against_ema is not null and
    away_ppda_for_ema is not null and
    away_ppda_against_ema is not null
    """
    
    return dwh_conn.export_to_pandas(v_sql)

def get_away_prediction_data(feature_type_filter, month_filter):
    
    v_sql = """
select
    football_match_his_lid,
    round(home_xg_against_ema,1)           home_xg_against_ema,
    round(away_xg_for_ema,1)               away_xg_for_ema
from
    betting_mart.ml_poisson_features
where
    --filter for iterator
    to_char(match_date,'yyyymm') = """ + str(month_filter) + """ and
    feature_type = '""" + str(feature_type_filter) + """' and
    home_goals_for_ema is not null and
    home_goals_against_ema is not null and
    away_goals_for_ema is not null and
    away_goals_against_ema is not null and
    home_xg_for_ema is not null and
    home_xg_against_ema is not null and
    away_xg_for_ema is not null and
    away_xg_against_ema is not null and
    home_shots_for_ema is not null and
    home_shots_against_ema is not null and
    away_shots_for_ema is not null and
    away_shots_against_ema is not null and
    home_sot_for_ema is not null and
    home_sot_against_ema is not null and
    away_sot_for_ema is not null and
    away_sot_against_ema is not null and
    home_corners_for_ema is not null and
    home_corners_against_ema is not null and
    away_corners_for_ema is not null and
    away_corners_against_ema is not null and
    home_deep_for_ema is not null and
    home_deep_against_ema is not null and
    away_deep_for_ema is not null and
    away_deep_against_ema is not null and
    home_ppda_for_ema is not null and
    home_ppda_against_ema is not null and
    away_ppda_for_ema is not null and
    away_ppda_against_ema is not null
    """
    
    return dwh_conn.export_to_pandas(v_sql)

## data function to get home and away data

In [9]:

import pandas
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor


#truncate result table
dwh_conn.execute('truncate table BETTING_DV.football_match_his_l_s_ml_poisson_expected_goals')


#get different feature types
df_feature_iterator = get_feature_types()

for index, row in df_feature_iterator.iterrows():
    print('Loop for feature type: ' + str(row['FEATURE_TYPE']))
    
    print('...getting data monthly loops for simulation')
    
    df_month_iterator = get_iterator_data(row['FEATURE_TYPE'])
    
    for index2, row2 in df_month_iterator.iterrows():
        
        print('...simulation for month ' + str(row2['ITERATOR']))
        
        print('.....getting training data')
        
        #get data to train home expGoals model
        df_home_train_data = get_home_training_data(row['FEATURE_TYPE'], row2['ITERATOR'])
        df_away_train_data = get_away_training_data(row['FEATURE_TYPE'], row2['ITERATOR'])
    
        #split features and pred classes
        df_home_train_x = df_home_train_data.iloc[:,:2]
        df_away_train_x = df_away_train_data.iloc[:,:2]
        df_train_home_y = df_home_train_data.iloc[:,2]
        df_train_away_y = df_away_train_data.iloc[:,2]
                
        print('......training models')
        
        print('......train home x: ' + str(df_home_train_x.shape))
        print('......train away x: ' + str(df_away_train_x.shape))
        print('......train home y: ' + str(df_train_home_y.shape))
        print('......train away y: ' + str(df_train_away_y.shape))
        
        
        model_home = XGBRegressor(
                                            n_estimators = 100,
                                            max_depth = 3,
                                            eta = 0.06
                                            ).fit(df_home_train_x, df_train_home_y)
        
        
        model_away = XGBRegressor(
                                            n_estimators = 100,
                                            max_depth = 2,
                                            eta = 0.06
                                            ).fit(df_away_train_x, df_train_away_y)


        print('......getting prediction data')
        
        #get data to train home expGoals model
        df_home_pred_data = get_home_prediction_data(row['FEATURE_TYPE'], row2['ITERATOR'])
        df_away_pred_data = get_away_prediction_data(row['FEATURE_TYPE'], row2['ITERATOR'])
        
        print('......df_home_pred_data: ' + str(df_home_pred_data.shape))
        print('......df_away_pred_data: ' + str(df_away_pred_data.shape))
        
        #do prediction
        df_home_pred_x = df_home_pred_data.iloc[:,1:3]
        df_away_pred_x = df_away_pred_data.iloc[:,1:3]
        
        print('......executing predition')  
    
        df_pred_home_expGoals = model_home.predict(df_home_pred_x)
        df_pred_away_expGoals = model_away.predict(df_away_pred_x)
    
        #create result data frame
        df_result_data = pd.DataFrame(df_home_pred_data.iloc[:,0:1])
        df_result_data['feature_type'] = 'xgboost/xg/' + str(row['FEATURE_TYPE'])
        df_result_data['home_exp_goals'] = df_pred_home_expGoals
        df_result_data['away_exp_goals'] = df_pred_away_expGoals
        df_result_data['interator_param'] = str(row2['ITERATOR'])


        print(df_result_data.head())
        
        print('......data gets written to the db')    
        dwh_conn.import_from_pandas(df_result_data,('BETTING_DV','football_match_his_l_s_ml_poisson_expected_goals'))
    

Loop for feature type: EMA15 COMB
...getting data monthly loops for simulation
...simulation for month 201608
.....getting training data
......training models
......train home x: (3590, 2)
......train away x: (3590, 2)
......train home y: (3590,)
......train away y: (3590,)
......getting prediction data
......df_home_pred_data: (100, 3)
......df_away_pred_data: (100, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID           feature_type  home_exp_goals  \
0  df74d6019d8ec8969da3b8a2d316d111  xgboost/xg/EMA15 COMB        1.450497   
1  733d356d163a75c04365593496d232e1  xgboost/xg/EMA15 COMB        1.172954   
2  acee56b53a1a434affc5cf8caedd2ef1  xgboost/xg/EMA15 COMB        1.512624   
3  6b2f710e0f9b6817cee287a160cfa86f  xgboost/xg/EMA15 COMB        1.370622   
4  fe382c3ac79fce6472a8d35927482d7f  xgboost/xg/EMA15 COMB        1.089548   

   away_exp_goals interator_param  
0        1.006276          201608  
1        1.195691          201608  
2        1.251918       

...simulation for month 201704
.....getting training data
......training models
......train home x: (4960, 2)
......train away x: (4960, 2)
......train home y: (4960,)
......train away y: (4960,)
......getting prediction data
......df_home_pred_data: (280, 3)
......df_away_pred_data: (280, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID           feature_type  home_exp_goals  \
0  45d9adc1ac85ebbc3ae282b4d0dea723  xgboost/xg/EMA15 COMB        1.634083   
1  8325fa391bcd8753c483e4b0e36c566f  xgboost/xg/EMA15 COMB        1.615328   
2  2b1217948b104826ac1ae44fc92f835c  xgboost/xg/EMA15 COMB        1.273721   
3  904cd484e4b3b00ec6b596b70c30a8ee  xgboost/xg/EMA15 COMB        1.437833   
4  295f4ef62dfa39d56fe2ed7db954fc51  xgboost/xg/EMA15 COMB        1.587717   

   away_exp_goals interator_param  
0        0.876036          201704  
1        1.292143          201704  
2        1.022490          201704  
3        1.254830          201704  
4        0.979855          2017

...simulation for month 201802
.....getting training data
......training models
......train home x: (6489, 2)
......train away x: (6489, 2)
......train home y: (6489,)
......train away y: (6489,)
......getting prediction data
......df_home_pred_data: (192, 3)
......df_away_pred_data: (192, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID           feature_type  home_exp_goals  \
0  5e9917fda3df324fedf6a04295e30c2d  xgboost/xg/EMA15 COMB        1.303780   
1  5eea0a2762973e4f5f685b5925f4ca98  xgboost/xg/EMA15 COMB        1.580187   
2  294ab534a68454aa256fbc49f7438136  xgboost/xg/EMA15 COMB        1.541207   
3  9e0520b3c9596568753131d5a1b7daf7  xgboost/xg/EMA15 COMB        1.752746   
4  e057d17ddda648f1a4ebab74653804c3  xgboost/xg/EMA15 COMB        1.499059   

   away_exp_goals interator_param  
0        1.465026          201802  
1        1.013797          201802  
2        1.076581          201802  
3        1.013797          201802  
4        1.355677          2018

...simulation for month 201812
.....getting training data
......training models
......train home x: (7860, 2)
......train away x: (7860, 2)
......train home y: (7860,)
......train away y: (7860,)
......getting prediction data
......df_home_pred_data: (249, 3)
......df_away_pred_data: (249, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID           feature_type  home_exp_goals  \
0  4552ce6db83c72975060f48d6893901c  xgboost/xg/EMA15 COMB        1.498117   
1  ad0d9c711b27657ab04b49d477d0793d  xgboost/xg/EMA15 COMB        1.699056   
2  462b7b50fa64886e8d93cf0b01b656c1  xgboost/xg/EMA15 COMB        1.390337   
3  cf6a150157e3e40b3fe5fcd9383143a9  xgboost/xg/EMA15 COMB        1.207483   
4  9892f50287f156de23f954402ca703e2  xgboost/xg/EMA15 COMB        1.206350   

   away_exp_goals interator_param  
0        1.637905          201812  
1        1.212908          201812  
2        0.977290          201812  
3        1.212908          201812  
4        1.329748          2018

...simulation for month 201910
.....getting training data
......training models
......train home x: (9370, 2)
......train away x: (9370, 2)
......train home y: (9370,)
......train away y: (9370,)
......getting prediction data
......df_home_pred_data: (165, 3)
......df_away_pred_data: (165, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID           feature_type  home_exp_goals  \
0  1fb31d814ca6f38ebf4b311d74d04ba8  xgboost/xg/EMA15 COMB        2.029915   
1  048fca762456c04edfeb4187c52b7919  xgboost/xg/EMA15 COMB        2.232482   
2  74171d939368be7ae22a703d33c55da0  xgboost/xg/EMA15 COMB        2.079391   
3  4dfe5158d80c5af72026430f938b0ad4  xgboost/xg/EMA15 COMB        1.157668   
4  e7733d20e21b86d90e04ebaf2b420be4  xgboost/xg/EMA15 COMB        1.294176   

   away_exp_goals interator_param  
0        1.196432          201910  
1        0.982283          201910  
2        0.831870          201910  
3        1.804523          201910  
4        1.058722          2019

...

...simulation for month 201809
.....getting training data
......training models
......train home x: (7331, 2)
......train away x: (7331, 2)
......train home y: (7331,)
......train away y: (7331,)
......getting prediction data
......df_home_pred_data: (226, 3)
......df_away_pred_data: (226, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID           feature_type  home_exp_goals  \
0  72ae7e7ffdf1140cb642ced7400fdd5a  xgboost/xg/EMA20 COMB        2.100882   
1  ea2bc487723cfe2fd8b7f12c6e6f4e49  xgboost/xg/EMA20 COMB        2.718265   
2  77b3d1f642756bedc00c5182c33b9273  xgboost/xg/EMA20 COMB        1.088956   
3  8355388fb61b834d7fa03da8e86c1bc9  xgboost/xg/EMA20 COMB        1.637281   
4  fed6fc1c1679fdd240068761df29e80b  xgboost/xg/EMA20 COMB        1.344037   

   away_exp_goals interator_param  
0        1.190274          201809  
1        0.859882          201809  
2        1.108459          201809  
3        1.619552          201809  
4        0.859882          2018

...simulation for month 201905
.....getting training data
......training models
......train home x: (8884, 2)
......train away x: (8884, 2)
......train home y: (8884,)
......train away y: (8884,)
......getting prediction data
......df_home_pred_data: (158, 3)
......df_away_pred_data: (158, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID           feature_type  home_exp_goals  \
0  546dfdb964ad6b4f2feb29326b41af74  xgboost/xg/EMA20 COMB        1.278234   
1  e208b4dd39680e9ccb30bde6c292ab37  xgboost/xg/EMA20 COMB        1.839750   
2  f959cd029526e03d0a056c5ef010f75b  xgboost/xg/EMA20 COMB        1.510529   
3  06f9d5da154b61a950bf2ebb9ce3f19e  xgboost/xg/EMA20 COMB        1.778225   
4  e03e0f758f6244d8e2c8ec9fff951394  xgboost/xg/EMA20 COMB        1.278234   

   away_exp_goals interator_param  
0        1.117621          201905  
1        1.179655          201905  
2        1.412463          201905  
3        2.048651          201905  
4        1.025013          2019

...simulation for month 202003
.....getting training data
......training models
......train home x: (10293, 2)
......train away x: (10293, 2)
......train home y: (10293,)
......train away y: (10293,)
......getting prediction data
......df_home_pred_data: (60, 3)
......df_away_pred_data: (60, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID           feature_type  home_exp_goals  \
0  98da06a8dcb879de42d7df0ddac017f8  xgboost/xg/EMA20 COMB        1.295018   
1  7dfb49ae68215726f5e4cefca04abf59  xgboost/xg/EMA20 COMB        3.101095   
2  2e4a8f62723e5cdec91e35a3d61bfdd8  xgboost/xg/EMA20 COMB        1.448396   
3  f12efde5297ebf3f2d4aa63b4e54085f  xgboost/xg/EMA20 COMB        2.132291   
4  e5e88e433e207e605bb186acabd99386  xgboost/xg/EMA20 COMB        1.928512   

   away_exp_goals interator_param  
0        1.152535          202003  
1        0.906108          202003  
2        1.121515          202003  
3        1.395692          202003  
4        1.460941          20

...simulation for month 202012
.....getting training data
......training models
......train home x: (11245, 2)
......train away x: (11245, 2)
......train home y: (11245,)
......train away y: (11245,)
......getting prediction data
......df_home_pred_data: (244, 3)
......df_away_pred_data: (244, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID           feature_type  home_exp_goals  \
0  0741c4cc4c27a235607ec5143b4a5356  xgboost/xg/EMA20 COMB        1.348653   
1  a79ef467383df1d0f6b4acef12f46f5a  xgboost/xg/EMA20 COMB        1.255157   
2  746da71c0b33775c166ae2f9a4eee1ba  xgboost/xg/EMA20 COMB        1.655230   
3  e4d372d7bd1712a4e8f13320e98d6c29  xgboost/xg/EMA20 COMB        2.641547   
4  4815051f51b1ea5314bae55ce317320b  xgboost/xg/EMA20 COMB        1.839166   

   away_exp_goals interator_param  
0        1.219375          202012  
1        1.020204          202012  
2        0.517045          202012  
3        1.383318          202012  
4        1.528658          

...simulation for month 202110
.....getting training data
......training models
......train home x: (12897, 2)
......train away x: (12897, 2)
......train home y: (12897,)
......train away y: (12897,)
......getting prediction data
......df_home_pred_data: (211, 3)
......df_away_pred_data: (211, 3)
......executing predition
             FOOTBALL_MATCH_HIS_LID           feature_type  home_exp_goals  \
0  a808c3a4540d5f91776402344ebfb8a4  xgboost/xg/EMA20 COMB        2.226772   
1  079def73a5cc60b5a3500ea4c7116c63  xgboost/xg/EMA20 COMB        1.654567   
2  73b27ee9e7b182beb98607ece12f14b8  xgboost/xg/EMA20 COMB        0.974422   
3  768135833d8e52cd175afb68c4f5bf21  xgboost/xg/EMA20 COMB        1.432950   
4  d0b610ed82674b5a0f3022d1a63fa596  xgboost/xg/EMA20 COMB        2.528389   

   away_exp_goals interator_param  
0        0.812946          202110  
1        1.144963          202110  
2        1.615187          202110  
3        1.071098          202110  
4        1.149019          